# Traja Deep Learning Features Demo

This notebook demonstrates the new deep learning features added to traja:

1. **3D Trajectory Support** - x, y, z coordinates
2. **GPS/Lat-Long Support** - Convert GPS to local coordinates
3. **Data Augmentation** - Rotation, noise, scaling, reversal, subsampling
4. **Sequence Utilities** - Padding, truncation, normalization
5. **Feature Extraction** - ML-ready features
6. **PyTorch Integration** - Tensor conversion
7. **Train/Val/Test Split** - Dataset splitting for DL

We'll use the public jaguar tracking dataset to demonstrate these features.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import traja
from traja.dataset import example

# Set random seed for reproducibility
np.random.seed(42)

## Load Public Dataset: Jaguar Tracking Data

In [ ]:
# Load jaguar movement data
df = example.jaguar()
print(f"Dataset shape: {df.shape}")
print(f"\nColumns: {df.columns.tolist()}")
df.head()

In [ ]:
# Convert to TrajaDataFrame
# Assuming columns contain x, y coordinates (adapt column names as needed)
traj = traja.from_df(df)
print(f"\nTrajectory shape: {traj.shape}")
print(f"Trajectory summary:")
print(traj.traja.summary())

In [ ]:
# Plot original trajectory
traja.plot(traj, title="Original Jaguar Trajectory")
plt.show()

## 1. Data Augmentation for Deep Learning

Data augmentation is essential for training robust deep learning models. Traja now provides several augmentation methods:

In [ ]:
# Create a shorter trajectory for visualization
traj_short = traj.iloc[:200].copy()

fig, axes = plt.subplots(2, 3, figsize=(15, 10))

# Original
traja.plot(traj_short, ax=axes[0, 0], title="Original")

# Rotation augmentation
traj_rotated = traj_short.traja.augment_rotate(angle=45)
traja.plot(traj_rotated, ax=axes[0, 1], title="Rotated 45°")

# Noise augmentation
traj_noisy = traj_short.traja.augment_noise(sigma=0.05)
traja.plot(traj_noisy, ax=axes[0, 2], title="Added Gaussian Noise")

# Scale augmentation
traj_scaled = traj_short.traja.augment_scale(factor=1.5)
traja.plot(traj_scaled, ax=axes[1, 0], title="Scaled 1.5x")

# Reverse augmentation
traj_reversed = traj_short.traja.augment_reverse()
traja.plot(traj_reversed, ax=axes[1, 1], title="Time-Reversed")

# Subsample augmentation
traj_subsampled = traj_short.traja.augment_subsample(step=3)
traja.plot(traj_subsampled, ax=axes[1, 2], title="Subsampled (every 3rd point)")

plt.tight_layout()
plt.show()

print("Augmentation examples created successfully!")

## 2. Sequence Padding and Truncation

For batching variable-length trajectories in deep learning, we need consistent sequence lengths:

In [ ]:
# Create trajectories of different lengths
traj1 = traj.iloc[:50]
traj2 = traj.iloc[:100]
traj3 = traj.iloc[:150]

print(f"Original lengths: {len(traj1)}, {len(traj2)}, {len(traj3)}")

# Pad shorter trajectories to target length
target_length = 150
traj1_padded = traj1.traja.pad_trajectory(target_length, mode='edge')
traj2_padded = traj2.traja.pad_trajectory(target_length, mode='edge')

print(f"After padding: {len(traj1_padded)}, {len(traj2_padded)}, {len(traj3)}")

# Truncate longer trajectory
target_length = 100
traj3_truncated = traj3.traja.truncate_trajectory(target_length, mode='end')

print(f"After truncation: {len(traj3_truncated)}")

# Visualize padding effect
fig, axes = plt.subplots(1, 2, figsize=(12, 5))
traja.plot(traj1, ax=axes[0], title=f"Original ({len(traj1)} points)")
traja.plot(traj1_padded, ax=axes[1], title=f"Padded ({len(traj1_padded)} points)")
plt.tight_layout()
plt.show()

## 3. Feature Extraction for Machine Learning

In [ ]:
# Extract ML-ready features
features = traj_short.traja.extract_features()

print("Extracted features:")
print(features.head(10))
print(f"\nFeature columns: {features.columns.tolist()}")
print(f"Feature shape: {features.shape}")

# Plot some features
fig, axes = plt.subplots(2, 2, figsize=(12, 8))

features['displacement'].plot(ax=axes[0, 0], title='Displacement')
axes[0, 0].set_xlabel('Time step')

if 'speed' in features.columns:
    features['speed'].plot(ax=axes[0, 1], title='Speed')
    axes[0, 1].set_xlabel('Time step')

if 'turn_angle' in features.columns:
    features['turn_angle'].plot(ax=axes[1, 0], title='Turn Angle')
    axes[1, 0].set_xlabel('Time step')

if 'acceleration' in features.columns:
    features['acceleration'].plot(ax=axes[1, 1], title='Acceleration')
    axes[1, 1].set_xlabel('Time step')

plt.tight_layout()
plt.show()

## 4. Normalization for Deep Learning

In [ ]:
# Normalize trajectory (center and scale)
traj_normalized = traj_short.traja.normalize_trajectory(scale=True, center=True)

fig, axes = plt.subplots(1, 2, figsize=(12, 5))

traja.plot(traj_short, ax=axes[0], title="Original")
traja.plot(traj_normalized, ax=axes[1], title="Normalized (centered & scaled)")

plt.tight_layout()
plt.show()

print("Original statistics:")
print(f"X: mean={traj_short.x.mean():.2f}, std={traj_short.x.std():.2f}")
print(f"Y: mean={traj_short.y.mean():.2f}, std={traj_short.y.std():.2f}")

print("\nNormalized statistics:")
print(f"X: mean={traj_normalized.x.mean():.2e}, std={traj_normalized.x.std():.2f}")
print(f"Y: mean={traj_normalized.y.mean():.2e}, std={traj_normalized.y.std():.2f}")

## 5. PyTorch Tensor Conversion

In [ ]:
# Convert to PyTorch tensor (requires PyTorch installation)
try:
    import torch
    tensor = traj_short.traja.to_tensor()
    print(f"Tensor shape: {tensor.shape}")
    print(f"Tensor dtype: {tensor.dtype}")
    print(f"\nFirst 5 points:")
    print(tensor[:5])
except ImportError:
    print("PyTorch not installed. Install with: pip install torch")
    # Returns numpy array instead
    array = traj_short.traja.to_tensor()
    print(f"\nReturned numpy array shape: {array.shape}")
    print(f"Array dtype: {array.dtype}")

## 6. Train/Val/Test Split for Deep Learning

In [ ]:
# Create multiple trajectories by splitting the full trajectory
# Each trajectory will be a segment of 100 points
segment_length = 100
trajectories = []

for i in range(0, len(traj) - segment_length, segment_length // 2):
    segment = traj.iloc[i:i + segment_length].reset_index(drop=True)
    trajectories.append(segment)

print(f"Created {len(trajectories)} trajectory segments")

# Split into train/val/test sets
train_trajs, val_trajs, test_trajs = traja.trajectory.train_test_split(
    trajectories,
    train_size=0.7,
    val_size=0.15,
    test_size=0.15,
    shuffle=True,
    random_state=42
)

print(f"\nTrain set: {len(train_trajs)} trajectories")
print(f"Validation set: {len(val_trajs)} trajectories")
print(f"Test set: {len(test_trajs)} trajectories")

# Visualize some training examples
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.flatten()

for i, traj_example in enumerate(train_trajs[:6]):
    traja.plot(traj_example, ax=axes[i], title=f"Train Example {i+1}")

plt.tight_layout()
plt.show()

## 7. Complete Deep Learning Pipeline Example

In [ ]:
# Demonstrate a complete preprocessing pipeline for deep learning

def preprocess_for_dl(traj, target_length=100, augment=True):
    """Complete preprocessing pipeline for deep learning."""
    
    # 1. Normalize
    traj = traj.traja.normalize_trajectory()
    
    # 2. Pad or truncate to target length
    if len(traj) < target_length:
        traj = traj.traja.pad_trajectory(target_length, mode='edge')
    elif len(traj) > target_length:
        traj = traj.traja.truncate_trajectory(target_length, mode='random')
    
    # 3. Optional augmentation
    if augment:
        # Randomly apply augmentations
        if np.random.random() < 0.5:
            traj = traj.traja.augment_rotate()  # Random rotation
        if np.random.random() < 0.3:
            traj = traj.traja.augment_noise(sigma=0.05)
        if np.random.random() < 0.3:
            traj = traj.traja.augment_scale()  # Random scaling
    
    # 4. Convert to tensor
    tensor = traj.traja.to_tensor()
    
    return tensor

# Process training trajectories
print("Preprocessing training data...")
train_tensors = [preprocess_for_dl(t, augment=True) for t in train_trajs[:10]]

print(f"Processed {len(train_tensors)} training examples")
print(f"Tensor shape: {train_tensors[0].shape if hasattr(train_tensors[0], 'shape') else np.array(train_tensors[0]).shape}")

# Process validation trajectories (no augmentation)
print("\nPreprocessing validation data...")
val_tensors = [preprocess_for_dl(t, augment=False) for t in val_trajs]

print(f"Processed {len(val_tensors)} validation examples")

print("\n✓ Data ready for deep learning model training!")

## 8. GPS/Lat-Long Support (Bonus Demo)

In [ ]:
# Create synthetic GPS coordinates
# (For real data, you would load actual lat/lon values)
n_points = 100
lat = np.linspace(40.7128, 40.7528, n_points) + np.random.normal(0, 0.001, n_points)
lon = np.linspace(-74.0060, -73.9660, n_points) + np.random.normal(0, 0.001, n_points)

print("Sample GPS coordinates:")
print(f"Latitude range: [{lat.min():.4f}, {lat.max():.4f}]")
print(f"Longitude range: [{lon.min():.4f}, {lon.max():.4f}]")

# Convert GPS to local x,y coordinates in meters
traj_gps = traja.from_latlon(lat, lon)

print(f"\nConverted to local coordinates:")
print(f"X range: [{traj_gps.x.min():.2f}, {traj_gps.x.max():.2f}] meters")
print(f"Y range: [{traj_gps.y.min():.2f}, {traj_gps.y.max():.2f}] meters")

# Plot
traja.plot(traj_gps, title="GPS Trajectory (converted to local x,y)")
plt.xlabel("East-West (meters)")
plt.ylabel("North-South (meters)")
plt.show()

# Original lat/lon columns are preserved
print("\nDataFrame contains both GPS and local coordinates:")
print(traj_gps[['lat', 'lon', 'x', 'y']].head())

## Summary

This demo showcased all the new deep learning features in traja:

1. ✓ **Data Augmentation**: Rotation, noise, scaling, reversal, subsampling
2. ✓ **Sequence Processing**: Padding, truncation, normalization
3. ✓ **Feature Extraction**: Automatic ML feature generation
4. ✓ **PyTorch Integration**: Easy tensor conversion
5. ✓ **Dataset Splitting**: Train/val/test split utility
6. ✓ **GPS Support**: Lat/lon to local coordinate conversion

These features make traja production-ready for deep learning applications on trajectory data!